# Signal Muting

This notebook demonstrates the signal muting functionality of SHF+ devices.
Signal muting can be used to effectively reduce the noise on idle signal lines.

## Python Imports

Import the necessary DSL classes from LabOne Q

In [ ]:
from laboneq.contrib.example_helpers.generate_device_setup import (
    generate_device_setup_qubits,
)
from laboneq.simple import *

## Device Setup

Create the device setup instance for the SHFQC instrument.


In [ ]:
# specify the number of qubits you want to use
number_of_qubits = 6

# generate the device setup and the qubit objects using a helper function
device_setup, qubits = generate_device_setup_qubits(
    number_qubits=number_of_qubits,
    pqsc=[{"serial": "DEV10001"}],
    hdawg=[{"serial": "DEV8001", "zsync": 0, "number_of_channels": 8, "options": None}],
    shfqc=[
        {
            "serial": "DEV12001",
            "zsync": 1,
            "number_of_channels": 6,
            "readout_multiplex": 6,
            "options": None,
        }
    ],
    include_flux_lines=True,
    server_host="localhost",
    setup_name=f"my_{number_of_qubits}_tuneable_qubit_setup",
)

In [ ]:
use_emulation = True

session = Session(device_setup)
session.connect(do_emulation=use_emulation)

## `automute` Calibration Option

Define the calibration function for the `acquire` and `measure` line of the SHFQA.

* The muting functionality is enabled on the `measure` line with the option `automute`.

In [ ]:
q0 = device_setup.logical_signal_groups["q0"]

q0.logical_signals["acquire_line"].calibration = SignalCalibration(
    oscillator=Oscillator(modulation_type=ModulationType.SOFTWARE, frequency=-2.5e8),
    local_oscillator=Oscillator(frequency=2e9),
    range=5,
)

q0.logical_signals["measure_line"].calibration = SignalCalibration(
    oscillator=Oscillator(modulation_type=ModulationType.SOFTWARE, frequency=-2.5e8),
    local_oscillator=Oscillator(frequency=2e9),
    automute=True,  # <-- Mute this output line when not playing pulses
    range=10,
)

## Experiment

Define an experiment that makes use of the muting functionality.

* The small amplitude of the play pulse operation allows us see the effect of the muting function better.
* While muting was already enabled by the calibration option `automute`, this functionality needs a sufficiently long delay in between pulses to become active.
* More specifically, a minimum delay of 280 ns is needed to mute a given signal line after a pulse has been played on it.

In [ ]:
exp = Experiment(
    signals=[
        ExperimentSignal("acquire", map_to=q0.logical_signals["acquire_line"]),
        ExperimentSignal("measure", map_to=q0.logical_signals["measure_line"]),
    ]
)
pulse = pulse_library.const(length=5e-7, amplitude=1)

with exp.acquire_loop_rt(count=2**5):
    with exp.section(uid="measure"):
        exp.play("measure", pulse=pulse, amplitude=0.01)
        exp.acquire("acquire", kernel=pulse, handle="h")
        exp.delay("acquire", 280e-9)
        exp.delay("measure", 280e-9)  # <-- muting active for delays >= 280 ns

compiled_exp = session.compile(exp)
print("\nSeqC:\n")
print(compiled_exp.scheduled_experiment.artifacts.src[0]["text"])

## Result

We are now ready to run the experiment and observe the muting of the signal on an oscilloscope.

The following wiring to the oscilloscope is used
* QA output signal
* QA marker signal


In [ ]:
_ = session.run(compiled_exp)

We see the behavior of the QA output and marker signal before and during the sequence of pulses played by
```
    exp.play("measure", pulse=pulse, amplitude=0.01)
```

and observe the following

* Without muting, we see the same noise level before and during the sequence of pulses 

* We see the muting function being active from the marker signal.

* When muting is active the noise level between individual pulses is reduced compared to before the pulse sequence.
